In [1]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# display plots in this notebook
%matplotlib inline

# for store the results
import pickle
import gzip

# for include pycode
import sys
import os
sys.path.insert(0,'../pycode')

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
caffe_root = '/opt/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
from caffe import layers as L
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import AesNet
from preprocess import utilities 

In [4]:
import utilsData
from sklearn.metrics import roc_auc_score, accuracy_score

# Generacion de train y test
Primero creamos dos txt con la informacion del fichero y de su clase

In [4]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))
data.loc[:,'id'] = data['id'].apply(str)
data.sort_values(['id'],inplace=True)
data.reset_index(inplace=True,drop=True)

classes = np.array(data.loc[:,'Class'])
ids = np.array(data.loc[:,'id'])
file_list = np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in ids])

In [6]:
np.random.seed(1000)
random_idx = np.random.randint(3, size=len(ids))

In [7]:
train_files = file_list[random_idx <= 1]
train_classes = classes[random_idx <= 1]

# Class balanced
random_balanced_indexes = utilities.balance_class(train_classes)
train_classes = train_classes[random_balanced_indexes]
train_files = train_files[random_balanced_indexes]

test_files = file_list[random_idx == 2]
test_classes = classes[random_idx == 2]

In [8]:
# This lines are for test the models with the same test as the networks
data_for_train_models = pd.DataFrame(data.loc[random_idx <= 1],copy=True)
data_for_train_models.reset_index(inplace=True)
data_for_train_models = data_for_train_models.loc[random_balanced_indexes]
train_indexes_for_models = np.array(data_for_train_models['index'])

data_for_test_models = pd.DataFrame(data.sort_values(['id']).loc[random_idx == 2],copy=True)
data_for_test_models.reset_index(inplace=True)
test_indexes_for_models = data_for_test_models['index']

In [9]:
pickle.dump(train_indexes_for_models, gzip.open( "../models/train_indexes_AesNet.pklz", "wb" ), 2)
pickle.dump(test_indexes_for_models, gzip.open( "../models/test_indexes_AesNet.pklz", "wb" ), 2)

In [10]:
train_indexes_for_models = pickle.load(gzip.open( "../models/train_indexes_AesNet.pklz", "rb" , 2))

In [11]:
df_train = pd.DataFrame(list(zip(train_files,train_classes)))
df_train.columns = ['image_filename', 'label']
df_train.to_csv('../models/train_partition_finetuning.txt', sep=' ', header=None, index=None)

df_test = pd.DataFrame(list(zip(test_files,test_classes)))
df_test.columns = ['image_filename', 'label']
df_test.to_csv('../models/test_partition_finetuning.txt', sep=' ', header=None, index=None)

## En este punto vamos a realizar la partición estandar de AVA
Son 230.000 casos de train y 20.000 de test

In [5]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))
data.loc[:,'id'] = data['id'].apply(str)
data.sort_values(['id'],inplace=True)
data.reset_index(inplace=True,drop=True)

classes = np.array(data.loc[:,'Class'])
ids = np.array(data.loc[:,'id'])
file_list = np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in ids])

In [6]:
np.random.seed(1000)
random_idx = np.random.randint(25, size=len(ids))

In [7]:
train_files = file_list[random_idx < 23]
train_classes = classes[random_idx < 23]

# Class balanced
random_balanced_indexes = utilities.balance_class(train_classes)
train_classes = train_classes[random_balanced_indexes]
train_files = train_files[random_balanced_indexes]

test_files = file_list[random_idx >= 23]
test_classes = classes[random_idx >= 23]

In [8]:
# This lines are for test the models with the same test as the networks
data_for_train_models = pd.DataFrame(data.loc[random_idx < 23],copy=True)
data_for_train_models.reset_index(inplace=True)
data_for_train_models = data_for_train_models.loc[random_balanced_indexes]
train_indexes_for_models = np.array(data_for_train_models['index'])

data_for_test_models = pd.DataFrame(data.loc[random_idx >= 23],copy=True)
data_for_test_models.reset_index(inplace=True)
test_indexes_for_models = data_for_test_models['index']

In [9]:
pickle.dump(train_indexes_for_models, gzip.open( "../models/train_indexes_AesNet_standard_AVA_balanced.pklz", "wb" ), 2)
pickle.dump(test_indexes_for_models, gzip.open( "../models/test_indexes_AesNet_standard_AVA_balanced.pklz", "wb" ), 2)

In [10]:
df_train = pd.DataFrame(list(zip(train_files,train_classes)))
df_train.columns = ['image_filename', 'label']

df_test = pd.DataFrame(list(zip(test_files,test_classes)))
df_test.columns = ['image_filename', 'label']

In [11]:
df_train.to_csv('../models/train_partition_finetuning_standard_AVA_balanced.txt', sep=' ', header=None, index=None)
df_test.to_csv('../models/test_partition_finetuning_standard_AVA_balanced.txt', sep=' ', header=None, index=None)

In [12]:
train_indexes_for_models = pickle.load(gzip.open( "../models/train_indexes_AesNet_standard_AVA_balanced.pklz", "rb" , 2))
df_train = pd.read_csv('../models/train_partition_finetuning_standard_AVA_balanced.txt', sep=' ', header=None)

In [19]:
data.loc[train_indexes_for_models]

,line,id,vote1,vote2,vote3,vote4,vote5,vote6,vote7,vote8,...,animal,architecture,cityscape,floral,fooddrink,landscape,portrait,stilllife,VotesMean,Class
2,75970,10002,5,14,36,60,35,29,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.243386,0
8,75909,10008,7,15,47,54,38,16,6,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.994595,0
9,75986,100082,0,2,7,25,65,79,68,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.409786,1
11,75935,10010,11,9,23,36,42,27,22,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.808743,0
12,75885,10011,3,4,11,26,38,42,36,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.705882,1
13,29196,10012,1,0,6,11,20,26,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,5.786517,1
15,29189,10014,3,2,5,23,27,17,8,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.966292,0
16,76122,100152,6,14,40,80,98,43,21,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.694268,0
19,75924,10017,10,13,30,49,38,28,7,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.380435,0
20,29219,10019,4,5,5,20,21,22,7,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.921348,0


In [17]:
df_train

,0,1
0,/home/frubio/AVA/AVADataset/10002.jpg,0
1,/home/frubio/AVA/AVADataset/10008.jpg,0
2,/home/frubio/AVA/AVADataset/100082.jpg,1
3,/home/frubio/AVA/AVADataset/10010.jpg,0
4,/home/frubio/AVA/AVADataset/10011.jpg,1
5,/home/frubio/AVA/AVADataset/10012.jpg,1
6,/home/frubio/AVA/AVADataset/10014.jpg,0
7,/home/frubio/AVA/AVADataset/100152.jpg,0
8,/home/frubio/AVA/AVADataset/10017.jpg,0
9,/home/frubio/AVA/AVADataset/10019.jpg,0


# Aqui comienza la chicha

In [ ]:
caffe.set_device(0)  # if we have multiple GPUs, pick the first one
#caffe.set_mode_cpu()

In [ ]:
weights = caffe_root + 'models/VGG-16/VGG_ILSVRC_16_layers.caffemodel'

In [ ]:
niter = 10  # number of iterations to train

# Reset style_solver as before.
style_solver_filename = aestheticNet.solver(aestheticNet.VGG16_aes_net(train=True,
                                                                       source_path='../models/%s_partition_finetuning.txt'),
                                            snapshot_pref = '../models/VGG16_AesNet',
                                            base_lr=0.001)
style_solver = caffe.get_solver(style_solver_filename)
style_solver.net.copy_from(weights)

In [ ]:
print ('Running solvers for %d iterations...' % niter)
solvers = [('pretrained', style_solver)]
loss, acc, weights = aestheticNet.run_solvers(niter, solvers)
print ('Done.')

train_loss = loss['pretrained']
train_acc = acc['pretrained']
style_weights = weights['pretrained']

# Delete solvers to save memory.
del style_solver, solvers

In [ ]:
os.rename(weights['pretrained'], "/home/frubio/AVA/aesthetic_finetuning.caffemodel")

# Testing

In [ ]:
model_def = aestheticNet.caffenet_only1aes_test()
model_weights = "../models/CaffeNet_only1Aes_iter_17000.caffemodel"

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [ ]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values

In [ ]:
# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

In [ ]:
data = pickle.load(gzip.open('../packages/AVA_info.pklz','rb',2))
data.loc[:,'id'] = data['id'].apply(str)
data.sort_values(['id'],inplace=True)
data.reset_index(inplace=True,drop=True)

test_cases = data.loc[pickle.load(gzip.open( "../models/test_indexes_AesNet_standard_AVA.pklz", "rb" , 2))]
test_files = np.array(['/home/frubio/AVA/AVADataset/{:}.jpg'.format(i) for i in test_cases['id']])
test_classes = np.array(test_cases['Class'])

In [ ]:
num_images = test_cases.shape[0]
output_prob = np.zeros(num_images)
batch_size = net.blobs['data'].data.shape[0]

num_forwards = np.ceil(num_images/batch_size).astype(int)

In [ ]:
for i in range(0,num_forwards):
    images_step = i*batch_size
    
    if (i == num_forwards - 1):
        images_to_process = num_images - images_step
    else:
        images_to_process = batch_size
            
    for j in range(0,images_to_process):
        # these lines is for those images that have four dimensions
        checked_image = caffe.io.load_image(test_files[j+images_step])
        if (len(checked_image.shape)==4):
            checked_image = checked_image[0]
                
        net.blobs['data'].data[j] = transformer.preprocess('data',checked_image)
        
    net.forward()
    output_prob[images_step:images_step+images_to_process] = net.blobs['probs'].data[0:images_to_process,1]

In [ ]:
results = {}
results['balanced'] = utilsData.balanced_accuracy(test_classes, output_prob)
results['AUC'] = roc_auc_score(test_classes, output_prob)
results['accuracy'] = accuracy_score(test_classes, (output_prob >= 0.5).astype(int))

In [ ]:
results

In [ ]:
pickle.dump(results, gzip.open( "../results/AesNet_VGG16.pklz", "wb" ), 2)